In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.utils import shuffle

In [2]:
titanic = pd.read_csv('titanic.csv')

In [3]:
def xavier_init(fan_in, fan_out, constant=1):
    low = -constant*np.sqrt(6/(fan_in+fan_out))
    high = constant*np.sqrt(6/(fan_in+fan_out))
    return tf.random_uniform((fan_in, fan_out), minval=low, maxval=high, dtype=tf.float32)

In [4]:
def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean)
        stdev = tf.sqrt(tf.reduce_mean(tf.square(var-mean)))
        tf.summary.scalar('stdev', stdev)
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.scalar('min', tf.reduce_min(var))
        tf.summary.histogram('histogram', var)

In [31]:
class dnn(object):
    def __init__(self, layers, n_input, n_classes, activation=tf.nn.relu, batch_size=50, learning_rate=0.01):
        ###we want to insert dropout too
        self.layers = layers
        self.n_input = n_input
        self.n_classes = n_classes
        self.activation = activation
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        
        self.x = tf.placeholder(tf.float32, [None, n_input])
        self.y = tf.placeholder(tf.int32, [None, ])
        
        self._forward_pass()
        self._cost()
        self._optimize()
        
        self.sess = tf.InteractiveSession()
        init = tf.global_variables_initializer()
        self.sess.run(init)
        
        self.merged = tf.summary.merge_all()
        self.writer = tf.summary.FileWriter('/Users/Blackbak/Documents/python_folder/tensorboard/', self.sess.graph)
        
    def _init_variables(self):
        self.variables = {}
        Mi = self.n_input
        for units, i in zip(self.layers, range(len(self.layers))):
            with tf.name_scope('layer_{}'.format(i)):
                Mo = units
                with tf.name_scope('weights'):
                    self.variables['layer_{}_weights'.format(i)] = tf.Variable(xavier_init(Mi, Mo), name='weights_{}'.format(i))
                    variable_summaries(self.variables['layer_{}_weights'.format(i)])
                with tf.name_scope('bias'):
                    self.variables['layer_{}_bias'.format(i)] = tf.Variable(tf.zeros(Mo), tf.float32, name = 'bias_{}'.format(i))
                    variable_summaries(self.variables['layer_{}_bias'.format(i)])
            Mi=Mo
        with tf.name_scope('output_layer'):
            with tf.name_scope('weights'):
                self.variables['layer_out_weights'] = tf.Variable(xavier_init(Mo, self.n_classes))
                variable_summaries(self.variables['layer_out_weights'])
            with tf.name_scope('bias'):
                self.variables['layer_out_bias'] = tf.Variable(tf.zeros(self.n_classes), tf.float32)
                variable_summaries(self.variables['layer_out_bias'])
                
    def _forward_pass(self):
        self._init_variables()
        layer_input = self.x
        for i in range(len(self.layers)):
            layer_input = self.activation(tf.add(tf.matmul(layer_input, self.variables['layer_{}_weights'.format(i)]),\
                                                self.variables['layer_{}_bias'.format(i)]))
        self.output = tf.nn.softmax(tf.add(tf.matmul(layer_input, self.variables['layer_out_weights']),
                                     self.variables['layer_out_bias']))

    
    def _cost(self):
        with tf.name_scope('cross_entropy'):
            self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=self.y, logits=self.output))
            variable_summaries(self.cost)
    
    def _optimize(self):
        with tf.name_scope('opt'):
            self.opt = tf.train.AdamOptimizer(learning_rate=self.learning_rate).minimize(self.cost)
    
    def _train_op(self, X, Y):
        opt, cost, summary = self.sess.run([self.opt, self.cost, self.merged], feed_dict={self.x : X, self.y : Y})
        #optimizer, summary = self.sess.run([self.opt, self.merged], feed_dict={self.x:X, self.y:Y})
        return cost, summary
    
    def predict(self, X):
        output = self.sess.run(self.output, feed_dict={self.x : X})
        
    def accuracy(self, X, Y):
        with tf.name_scope('accuracy'):
            correct_predictions = tf.equal(tf.argmax(self.predict(X=X),1),tf.argmax(Y, 1))
            accuracy = tf.reduce_mean(correct_predictions)
            tf.summary.scalar('accuracy', accuracy)
        return accuracy
        
    
    def train(self, X, Y, epochs=100):
        ####need to specify what is the format of X and Y
        n_samples = X.shape[0]
        n_batches = int(n_samples/self.batch_size)
        for epoch in range(epochs):
            X,Y = shuffle(X, Y)
            avg_cost = 0
            avg_accuracy = 0
            for batch in range(0,n_samples,self.batch_size):
                ###train_X = X[batch_size*batch:batch_size*(batch+1)]
                ###train_Y = Y[batch_size*batch:batch_size*(batch+1)]
                train_X = X[batch:batch+self.batch_size,:]
                train_Y = Y[batch:batch+self.batch_size]
                tf.cast(train_X, tf.float32)
                tf.cast(train_X, tf.int32)
                print('X', train_X)
                cost, summary = self._train_op(X=train_X, Y=train_Y)
                avg_cost += cost
                avg_accuracy += self.accuracy(X=train_X, Y=train_Y)
            avg_cost = avg_cost/n_batches
            avg_accuracy = avg_accuracy/n_batches
            self.writer.add_summary(summary, epoch)
            #### dunno if accuracy is going to get tracked self.writer.add_summary()
                
                
            

In [6]:
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [13]:
titanic = titanic.dropna()
Y = np.array(titanic['Survived'], dtype=int)
X = np.array(titanic[['Pclass', 'Age', 'Fare', 'SibSp']], dtype=float)

In [32]:
ff = dnn(layers = [10,10], n_input=X.shape[1], n_classes=1)

In [33]:
ff.train(X=X, Y=Y)

X [[   1.       48.       76.7292    1.    ]
 [   1.       48.       52.        1.    ]
 [   1.       38.       90.        1.    ]
 [   1.       18.       79.65      0.    ]
 [   1.       16.       39.4       0.    ]
 [   1.       27.       76.7292    0.    ]
 [   1.       31.       50.4958    0.    ]
 [   1.       58.       29.7       0.    ]
 [   1.       56.       35.5       0.    ]
 [   1.       17.       57.        1.    ]
 [   1.       22.       55.        0.    ]
 [   1.       21.       77.2875    0.    ]
 [   1.       29.       30.        0.    ]
 [   1.       35.       83.475     1.    ]
 [   1.       37.       52.5542    1.    ]
 [   2.       34.       10.5       0.    ]
 [   1.       64.      263.        1.    ]
 [   1.       58.       26.55      0.    ]
 [   1.       36.       78.85      1.    ]
 [   1.       45.       26.55      0.    ]
 [   1.       52.       78.2667    1.    ]
 [   2.        4.       39.        2.    ]
 [   1.       30.       56.9292    0.    ]
 [   1.  

InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder' with dtype float
	 [[Node: Placeholder = Placeholder[dtype=DT_FLOAT, shape=[], _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'Placeholder', defined at:
  File "/Users/Blackbak/Applications/anaconda/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/Blackbak/Applications/anaconda/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/Blackbak/Applications/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/Users/Blackbak/Applications/anaconda/lib/python3.5/site-packages/traitlets/config/application.py", line 653, in launch_instance
    app.start()
  File "/Users/Blackbak/Applications/anaconda/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 474, in start
    ioloop.IOLoop.instance().start()
  File "/Users/Blackbak/Applications/an